In [29]:
import subprocess
import os

from datetime import datetime, timedelta
import requests as r
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import random
from osgeo import gdal
import re
from tqdm import tqdm
import shutil
import calendar

import rasterio as rio
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy
from rasterio.plot import show

import pyproj
from pyproj import Proj

from shapely.ops import transform
from shapely.geometry import Polygon

import time

# import geoviews as gv
from cartopy import crs
# import hvplot.xarray
# import holoviews as hv
# gv.extension('bokeh', 'matplotlib')

import multiprocessing
from multiprocessing.pool import ThreadPool
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
%cd /scratch/user/anshulya
%ls

/scratch/user/anshulya
383_LAKE_MEAD.tif                full_processing_driver1.sh*
800_res.geojson                  full_processing_driver2.sh*
DAACDataDownload.py              full_processing_driver3.sh*
LAKE_MEAD_dem.tif                full_processing_driver4.sh*
PIP_CACHE/                       full_processing_driver5.sh*
WT_432.geojson                   hls_data/
area_estimation_driver.sh*       hls_job1.sh
area_job1.sh                     hls_job2.sh
blitz_script_1.sh*               hls_job3.sh
blitz_script_2.sh*               hls_job4.sh
blitz_script_3.sh*               in-situ_data.csv
blitz_script_4.sh*               jrc_data/
blitz_script_5.sh*               jupyter_notebooks/
blitz_script_6.sh*               lake_rf_new.pkl
clipped_data_creation.py         lake_rf_new_L30.pkl
clipped_raster.tif               log_files/
compare_stats.py                 max_img_extent.py
cubesat_data/                    ondemand/
cubesat_rf.pkl                   ondemand-faster/
delete_folders

In [3]:
def preprocess_reservoir_shp(fname):
  reservoirs = gp.read_file(fname)

  reservoirs.loc[107,'name'] = 'Farmer_s Creek Reservoir'

  t = ['/' in s for s in reservoirs['name']]
  i = 0
  res_names = ['Cox Lake', 'Eagle Nest Lake_Manor Lake', 'Lake Ballinger_Lake Moonen', 'Lake Olney_Lake Cooper', 'Mustang Lake']
  for res_idx in [77, 93, 183, 249, 313]:
    reservoirs.loc[res_idx, 'name'] = res_names[i]
    i = i + 1

  i = 0
  res_names = ['Lake Gonzales','Clear Lake Wa']
  for res_idx in [218, 70]:
    reservoirs.loc[res_idx, 'name'] = res_names[i]
    i = i + 1

  for i in range(809):
    string = reservoirs.loc[i, 'name']
    regex = re.compile('[@!#$%^&*()<>?/\|}{~]')
    # Pass the string in search
    # method of regex object.
    if(regex.search(string) == None):
      continue
    else:
      print(i,string)

  return reservoirs

In [4]:
fname = '800_res.geojson'
reservoirs = preprocess_reservoir_shp(fname)
reservoirs

,name,source,polygon_area(km2),grand_id,latitude,longitude,hylake_id,polygon_source,lake_type,lake_area,...,vol_src,depth_avg,dis_avg,res_time,elevation,slope_100,wshd_area,pour_long,pour_lat,geometry
0,Abiquiu Reservoir,BOR,15.690495,606.0,36.237769,-106.428630,9323.0,SWBD,2.0,15.69,...,2.0,107.599998,8.319,2349.3,1901.0,7.32,5546.8,-106.428630,36.237769,"POLYGON ((-106.47125 36.28749, -106.47128 36.2..."
1,Agate Reservoir,BOR,0.642683,-999.0,42.415300,-122.773300,1051060.0,SWBD,1.0,0.64,...,3.0,4.700000,0.417,84.6,454.0,3.23,34.9,-122.772917,42.413712,"POLYGON ((-122.76830 42.40168, -122.76837 42.4..."
2,Lake Alan Henry,TWDB,4.889970,1165.0,33.060408,-101.047925,113495.0,SWBD,2.0,4.89,...,2.0,89.400002,0.368,13753.6,674.0,3.03,2571.3,-101.047925,33.060408,"POLYGON ((-101.05109 33.06457, -101.05124 33.0..."
3,Alcoa Lake,TWDB,3.013936,1290.0,30.574558,-97.049735,113857.0,SWBD,2.0,3.02,...,2.0,7.800000,0.118,2295.2,144.0,1.30,21.1,-97.049735,30.574558,"POLYGON ((-97.07658 30.58325, -97.07668 30.583..."
4,Alcova,BOR,8.100403,415.0,42.543117,-106.727083,111277.0,SWBD,2.0,8.09,...,2.0,4.700000,73.335,5.9,1673.0,6.03,27437.2,-106.727083,42.543117,"POLYGON ((-106.77364 42.51554, -106.77200 42.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,Pool C,HYDROLAKES,5.413476,1856.0,34.260518,-88.426571,113177.0,SWBD,2.0,5.42,...,2.0,3.000000,12.705,14.8,80.0,1.51,575.4,-88.426571,34.260518,"POLYGON ((-88.41598 34.32292, -88.41568 34.323..."
805,Pool B,HYDROLAKES,9.600323,1865.0,34.065793,-88.422511,113208.0,SWBD,2.0,9.60,...,2.0,2.400000,66.231,4.1,74.0,2.00,2897.7,-88.422511,34.065793,"POLYGON ((-88.38646 34.12701, -88.38514 34.129..."
806,Aberdeen Lake,HYDROLAKES,5.645277,1868.0,33.832243,-88.519826,113253.0,SWBD,2.0,5.65,...,2.0,6.900000,135.363,3.3,56.0,1.16,5697.5,-88.519826,33.832243,"POLYGON ((-88.53134 33.88381, -88.53311 33.884..."
807,Columbus Lake,HYDROLAKES,9.011978,1876.0,33.519852,-88.488172,113327.0,SWBD,2.0,9.02,...,2.0,8.100000,272.854,3.1,48.0,0.92,11599.2,-88.488172,33.519852,"POLYGON ((-88.51374 33.55789, -88.51704 33.557..."


In [9]:
filenames = os.listdir('results')

# Lists to hold the sorted filenames
f_files = []
other_files = []

# Iterate through each filename and sort them
for filename in filenames:
    if '_f' in filename:
        f_files.append(filename)
    else:
        other_files.append(filename)

done_idx = []
for f in f_files:
    done_idx.append(int(f.split('_')[0]))

In [10]:
done_idx.sort()

In [11]:
in_situ_df = pd.read_csv('in-situ_data.csv')
in_situ_df

,Unnamed: 0,RES_NAME,date,stage_m,area_m2,depth_m,datum,qflag
0,0,ABIQUIU LAKE,2016-01-01,1895.8560,16304798.94,13.910954,NGVD29,S
1,1,ABIQUIU LAKE,2016-01-02,1895.8560,16304798.94,13.910954,NGVD29,S
2,2,ABIQUIU LAKE,2016-01-03,1895.8560,16304798.94,13.910954,NGVD29,S
3,3,ABIQUIU LAKE,2016-01-04,1895.8560,16304798.94,13.910954,NGVD29,S
4,4,ABIQUIU LAKE,2016-01-05,1895.8560,16304798.94,13.910954,NGVD29,S
...,...,...,...,...,...,...,...,...
542359,542359,WILLOW CREEK RESERVOIR MT,2021-12-27,1262.4816,5916509.32,6.639720,RPVD,S
542360,542360,WILLOW CREEK RESERVOIR MT,2021-12-28,1262.4816,5916509.32,6.639720,RPVD,S
542361,542361,WILLOW CREEK RESERVOIR MT,2021-12-29,1262.4816,5916509.32,6.639720,RPVD,S
542362,542362,WILLOW CREEK RESERVOIR MT,2021-12-30,1262.4816,5916509.32,6.639720,RPVD,S


In [38]:
for res_idx in [1,19,23,25,79,24,100,112,122,138,157,237]:
    df_final= pd.read_csv('results/'+str(res_idx)+'_f.csv')
    df_final['datetime'] = pd.to_datetime(df_final['datetime'])
    df_final = df_final[df_final['days']>0]

    mask = in_situ_df['RES_NAME'].str.lower().apply(lambda x: any(lake.lower() in x for lake in [reservoirs.loc[res_idx,'name']]))
    in_situ = in_situ_df[mask]
    # in_situ = in_situ_df[in_situ_df['RES_NAME']=='LAKE MEAD']
    # in_situ['date'] = pd.to_datetime(in_situ['date'])

    print(res_idx, in_situ.iloc[0,1].title())

    # merged_df = pd.merge(in_situ[['date','area_m2']], df_final[['datetime','lake_area','ice_cover','cloud_cover']], left_on='date', right_on='datetime', how='inner')
    # merged_df['in_situ_area'] = merged_df['area_m2']/10**6

    # # Calculate metrics
    # mae = mean_absolute_error(merged_df['in_situ_area'], merged_df['lake_area'])
    # rmse = np.sqrt(mean_squared_error(merged_df['in_situ_area'], merged_df['lake_area']))
    # mbd = np.mean(merged_df['lake_area'] - merged_df['in_situ_area'])
    # mpe = np.mean((merged_df['lake_area'] - merged_df['in_situ_area']) / merged_df['in_situ_area']) * 100
    # mape = np.mean(np.abs((merged_df['lake_area'] - merged_df['in_situ_area']) / merged_df['in_situ_area'])) * 100
    # r2 = r2_score(merged_df['in_situ_area'], merged_df['lake_area']+mae)
    # relative_bias = (mbd / np.mean(merged_df['in_situ_area'])) * 100
    
    # # Calculate Nash-Sutcliffe Efficiency (NSE)
    # observed = merged_df['in_situ_area']
    # simulated = merged_df['lake_area']+mae
    # nse = 1 - (np.sum((observed - simulated)**2) / np.sum((observed - np.mean(observed))**2))
    
    # # Calculate Pearson Correlation Coefficient
    # pearson_corr = merged_df['in_situ_area'].corr(merged_df['lake_area'])

    # daily_df = pd.read_csv('results/'+str(res_idx)+'.csv')
    # daily_df['datetime'] = pd.to_datetime(daily_df['date'], format='%Y%j')
    # daily_df = daily_df[daily_df['datetime']<datetime(2022,1,1)]

    # # Set 'datetime' column as the index if it's not already
    # daily_df.set_index('datetime', inplace=True)
    # weekly_df = daily_df.resample('W').mean()  # You can change 'sum' to another aggregation function as needed
    # weekly_df.reset_index(inplace=True)
    # weekly_df.loc[weekly_df['cloud_cover']>100,'cloud_cover'] = 100

    # x1 = merged_df['in_situ_area'].quantile(0)
    # x2 = merged_df['in_situ_area'].quantile(1)
    # dx = (x2-x1)/100

    # fig, ax1 = plt.subplots(figsize=(7.5,2.5))
    
    # # Plot the first line on the main axis
    # ax1.plot(merged_df['datetime'], merged_df['lake_area'], marker='o', markersize=4, markerfacecolor='none', 
    #          markeredgecolor='black', alpha=0.5, linestyle='None',color='#ADD8E6',label='HLS estimates')
    # ax1.plot(merged_df['datetime'],merged_df['in_situ_area'],linewidth=3,alpha=0.75,label='in-situ data',color='blue')
    # ax1.set_ylabel('Lake Area (in km$^2$)',fontsize=12)
    
    # ax1.text(datetime(2017,1,1),x2+7.5*dx,
    #          '\u03C1 = {:.2f} \tR$^2$ = {:.2f} \tMAE = {:.2f} km$^2$ ({:.0f}%)'.format(pearson_corr,nse,mae,mape),
    #          fontsize=12)
    # ax1.set_ylim([x1-35*dx,x2+25*dx])
    
    # # Changing font size of x and y-tick labels for ax1
    # ax1.tick_params(axis='both', which='major', labelsize=11)
    
    # # Create a twin axis for the second line
    # ax2 = ax1.twinx()
    
    # # Create weekly bar plots for ice cover and cloud cover
    # bar_width = 7  # Width of each weekly bar in days
    # bar_positions = weekly_df['datetime']  # Start date of each week
    # ax2.bar(bar_positions, weekly_df['cloud_cover'], width=bar_width, alpha=1, color='#b9bbb6', label='Cloud Cover')
    # ax2.bar(bar_positions, weekly_df['ice_cover'], width=bar_width, alpha=1, color='#2494ae', label='Ice Cover')
    
    # #'#2494ae'
    
    # # Set the range for the twin axis
    # ax2.set_ylim(0, 1000)
    
    # # Set the y-axis label and change its color
    # ylabel = ax2.set_ylabel('Ice/Cloud cover (%)',fontsize=11)
    # ylabel.set_color('#828282')  # Change the color of the y-axis label
    
    # # Adjust the position of the y-axis label
    # ax2.yaxis.set_label_coords(1.02,0.57)  # Adjust the x position to move the label closer to the axis
    
    # # Manually set y-axis ticks up to 100
    # y_ticks = [tick for tick in range(0, 101, 100)]
    # ax2.set_yticks(y_ticks)
    
    # # plt.xlim([datetime(2016,3,1),datetime(2022,2,1)])
    # # ax2.legend(fontsize=12, bbox_to_anchor=(0.51, -0.1))
    # # ax2.legend(fontsize=12, bbox_to_anchor=(0.51, -0.1))
    # plt.title(in_situ.iloc[0,1].title(),fontsize=16)
    # plt.show()


1 Agate Reservoir
19 Beardsley Lake
23 Beulah Reservoir
25 Big Sandy Reservoir
79 Crawford Reservoir
24 Bighorn Lake
100 Elephant Butte Reservoir
112 Fontenelle Reservoir
122 Gerber Reservoir
138 Heron Reservoir
157 Jackson Lake
237 Lake Mead
